In [1]:
pwd

Path                                             
----                                             
C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell

## To build the project

We really just need to `dotnet restore` and `dotnet build` ...

But in order to ship something, we need to `publish` it:

In [1]:
dotnet publish -f netcoreapp2.0 -c Release
dotnet publish -f net462 -c Release

Microsoft (R) Build Engine version 15.3.117.23532
Copyright (C) Microsoft Corporation. All rights reserved.

  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\Jupyter.dll
  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\publish\
  PowerShell-Kernel -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\PowerShell-Kernel.dll
  PowerShell-Kernel -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\netcoreapp2.0\publish\
Microsoft (R) Build Engine version 15.3.117.23532
Copyright (C) Microsoft Corporation. All rights reserved.

  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\net462\Jupyter.dll
  Jupyter -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\net462\publish\
  PowerShell-Kernel -> C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release\net462\PowerShell-Kernel.exe
  PowerShell-Kernel -> C:\Users\Jo

In [2]:
mkdir Output\Release\Jupyter-PowerShell -force | sl
move-item ..\net462\publish PowerShell-Full
move-item ..\netcoreapp2.0\publish PowerShell-Core
copy-item ..\..\..\Install.ps1 .
cd ..
Microsoft.PowerShell.Archive\Compress-Archive Jupyter-PowerShell Jupyter-PowerShell.zip

In [3]:
ls

Directory: C:\Users\Joel\Projects\Jupyter\Jupyter-PowerShell\Output\Release


Mode                LastWriteTime         Length Name                                                                   
----                -------------         ------ ----                                                                   
d-----        6/29/2017  10:36 PM                Jupyter-PowerShell                                                     
d-----        6/29/2017  10:36 PM                net462                                                                 
d-----        6/29/2017  10:36 PM                netcoreapp2.0                                                          
-a----        6/29/2017  10:36 PM       14567990 Jupyter-PowerShell.zip